# Bài 1: Dịch máy thần kinh

Chào mừng bạn đến với bài tập đầu tiên của Khóa 4. Tại đây, bạn sẽ xây dựng mô hình dịch máy thần kinh từ tiếng Anh sang tiếng Bồ Đào Nha (NMT) bằng cách sử dụng mạng Bộ nhớ ngắn hạn dài (LSTM) một cách chú ý. Dịch máy là một nhiệm vụ quan trọng trong xử lý ngôn ngữ tự nhiên và có thể hữu ích không chỉ trong việc dịch ngôn ngữ này sang ngôn ngữ khác mà còn giúp phân biệt nghĩa của từ (ví dụ: xác định xem từ "ngân hàng" ám chỉ ngân hàng tài chính hay vùng đất ven sông) . Việc triển khai điều này chỉ bằng cách sử dụng Mạng thần kinh tái phát (RNN) với LSTM có thể hoạt động đối với các câu có độ dài từ ngắn đến trung bình nhưng có thể dẫn đến biến mất độ dốc đối với các chuỗi rất dài. Để giải quyết vấn đề này, bạn sẽ thêm một cơ chế chú ý để cho phép bộ giải mã truy cập tất cả các phần có liên quan của câu đầu vào bất kể độ dài của nó. Khi hoàn thành nhiệm vụ này, bạn sẽ:

- Triển khai hệ thống mã hóa-giải mã một cách chu đáo
- Xây dựng mô hình NMT từ đầu bằng Tensorflow
- Tạo bản dịch bằng cách sử dụng giải mã Tham lam và Rủi ro Bayes Tối thiểu (MBR)

## Mục lục
- [1 - Data Preparation](#1)
- [2 - NMT model with attention](#2)
- [Exercise 1 - Encoder](#ex1)
- [Exercise 2 - CrossAttention](#ex2)
- [Exercise 3 - Decoder](#ex3)
- [Exercise 4 - Translator](#ex4)
- [3 - Training](#3)
- [4 - Using the model for inference ](#4)
- [Exercise 5 - translate](#ex5)
- [5 - Minimum Bayes-Risk Decoding](#5)
- [Exercise 6 - rouge1_similarity](#ex6)
- [Exercise 7 - average_overlap](#ex7)


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Setting this env variable prevents TF warnings from showing up

import numpy as np
import tensorflow as tf
from collections import Counter
from utils import (sentences, train_data, val_data, english_vectorizer, portuguese_vectorizer, 
                   masked_loss, masked_acc, tokens_to_text)

In [2]:
import w1_unittest

<a tên="1"></a>
## 1. Chuẩn bị dữ liệu

Các bit xử lý trước văn bản đã được xử lý (nếu bạn quan tâm đến vấn đề này, hãy nhớ kiểm tra tệp `utils.py`). Các bước thực hiện có thể tóm tắt như sau:

- Đọc dữ liệu thô từ file văn bản
- Làm sạch dữ liệu (sử dụng chữ thường, thêm khoảng trắng xung quanh dấu câu, cắt bớt khoảng trắng, v.v.)
- Chia nó thành tập huấn luyện và xác nhận
- Thêm dấu đầu câu và dấu cuối câu vào mỗi câu
- Token hóa các câu
- Tạo tập dữ liệu Tensorflow từ các câu được mã hóa

Hãy dành chút thời gian để kiểm tra các câu thô:

In [3]:
portuguese_sentences, english_sentences = sentences

print(f"English (to translate) sentence:\n\n{english_sentences[-5]}\n")
print(f"Portuguese (translation) sentence:\n\n{portuguese_sentences[-5]}")

English (to translate) sentence:

No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.

Portuguese (translation) sentence:

Não importa o quanto você tenta convencer os outros de que chocolate é baunilha, ele ainda será chocolate, mesmo que você possa convencer a si mesmo e poucos outros de que é baunilha.


Các câu thô bạn không có nhiều sử dụng nên hãy xóa chúng đi để tiết kiệm bộ nhớ:

In [4]:
del portuguese_sentences
del english_sentences
del sentences

Lưu ý rằng bạn đã nhập `english_vectorizer` và `tiếng Bồ Đào Nha_vectorizer` từ `utils.py`. Chúng được tạo bằng [tf.keras.layers.TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) và chúng cung cấp các tính năng thú vị như cách trực quan hóa từ vựng và chuyển đổi văn bản thành id được mã hóa và ngược lại. Trên thực tế, bạn có thể kiểm tra mười từ đầu tiên của từ vựng cho cả hai ngôn ngữ:

In [5]:
print(f"First 10 words of the english vocabulary:\n\n{english_vectorizer.get_vocabulary()[:10]}\n")
print(f"First 10 words of the portuguese vocabulary:\n\n{portuguese_vectorizer.get_vocabulary()[:10]}")

First 10 words of the english vocabulary:

['', '[UNK]', '[SOS]', '[EOS]', '.', 'tom', 'i', 'to', 'you', 'the']

First 10 words of the portuguese vocabulary:

['', '[UNK]', '[SOS]', '[EOS]', '.', 'tom', 'que', 'o', 'nao', 'eu']


Lưu ý rằng 4 từ đầu tiên được dành riêng cho những từ đặc biệt. Theo thứ tự, đó là:

- chuỗi trống
- một mã thông báo đặc biệt để đại diện cho một từ chưa biết
- một mã thông báo đặc biệt để thể hiện sự bắt đầu của một câu
- một mã thông báo đặc biệt để thể hiện sự kết thúc của câu

Bạn có thể xem có bao nhiêu từ trong từ vựng bằng cách sử dụng phương thức `vocabulary_size`:

In [6]:
# Size of the vocabulary
vocab_size_por = portuguese_vectorizer.vocabulary_size()
vocab_size_eng = english_vectorizer.vocabulary_size()

print(f"Portuguese vocabulary is made up of {vocab_size_por} words")
print(f"English vocabulary is made up of {vocab_size_eng} words")

Portuguese vocabulary is made up of 12000 words
English vocabulary is made up of 12000 words


Bạn có thể xác định các đối tượng [tf.keras.layers.StringLookup](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) sẽ giúp bạn ánh xạ từ từ sang id và ngược lại. Thực hiện điều này đối với từ vựng tiếng Bồ Đào Nha vì điều này sẽ hữu ích sau này khi bạn giải mã các dự đoán từ mô hình của mình:

In [7]:
# This helps you convert from words to ids
word_to_id = tf.keras.layers.StringLookup(
    vocabulary=portuguese_vectorizer.get_vocabulary(), 
    mask_token="", 
    oov_token="[UNK]"
)

# This helps you convert from ids to words
id_to_word = tf.keras.layers.StringLookup(
    vocabulary=portuguese_vectorizer.get_vocabulary(),
    mask_token="",
    oov_token="[UNK]",
    invert=True,
)

Hãy dùng thử để tìm các mã thông báo đặc biệt và một từ ngẫu nhiên:

In [8]:
unk_id = word_to_id("[UNK]")
sos_id = word_to_id("[SOS]")
eos_id = word_to_id("[EOS]")
baunilha_id = word_to_id("baunilha")

print(f"The id for the [UNK] token is {unk_id}")
print(f"The id for the [SOS] token is {sos_id}")
print(f"The id for the [EOS] token is {eos_id}")
print(f"The id for baunilha (vanilla) is {baunilha_id}")

The id for the [UNK] token is 1
The id for the [SOS] token is 2
The id for the [EOS] token is 3
The id for baunilha (vanilla) is 7079


Cuối cùng hãy xem dữ liệu sẽ được đưa vào mạng lưới thần kinh trông như thế nào. Cả `train_data` và `val_data` đều thuộc loại `tf.data.Dataset` và đã được sắp xếp theo lô gồm 64 ví dụ. Để lấy lô đầu tiên ra khỏi tập dữ liệu tf, bạn có thể sử dụng phương thức `take`. Để lấy ví dụ đầu tiên trong lô, bạn có thể cắt tensor và sử dụng phương thức `numpy` để in đẹp hơn:

In [9]:
for (to_translate, sr_translation), translation in train_data.take(1):
    print(f"Tokenized english sentence:\n{to_translate[0, :].numpy()}\n\n")
    print(f"Tokenized portuguese sentence (shifted to the right):\n{sr_translation[0, :].numpy()}\n\n")
    print(f"Tokenized portuguese sentence:\n{translation[0, :].numpy()}\n\n")

Tokenized english sentence:
[   2  210    9  146  123   38    9 1672    4    3    0    0    0    0]


Tokenized portuguese sentence (shifted to the right):
[   2 1085    7  128   11  389   37 2038    4    0    0    0    0    0
    0]


Tokenized portuguese sentence:
[1085    7  128   11  389   37 2038    4    3    0    0    0    0    0
    0]




Có một vài chi tiết quan trọng cần chú ý.

- Phần đệm đã được áp dụng cho các tensor và giá trị được sử dụng cho việc này là 0
- Mỗi ví dụ gồm 3 tensor khác nhau:
- Câu cần dịch
- Dịch chuyển sang phải
- Bản dịch

Hai cái đầu tiên có thể được coi là các tính năng, trong khi cái thứ ba là mục tiêu. Bằng cách này, mô hình của bạn có thể thực hiện Buộc giáo viên như bạn đã thấy trong các bài giảng.

Bây giờ là lúc để bắt đầu viết mã!

<a tên="2"></a>
##2. Mô hình NMT có sự chú ý

Mô hình bạn sẽ xây dựng sử dụng kiến ​​trúc bộ mã hóa-giải mã. Mạng thần kinh tái phát (RNN) này lấy phiên bản mã hóa của một câu trong bộ mã hóa của nó, sau đó chuyển nó đến bộ giải mã để dịch. Như đã đề cập trong các bài giảng, chỉ cần sử dụng mô hình tuần tự thông thường với LSTM sẽ hoạt động hiệu quả đối với các câu ngắn và trung bình nhưng sẽ bắt đầu kém hiệu quả đối với các câu dài hơn. Bạn có thể hình dung nó giống như hình bên dưới, trong đó tất cả ngữ cảnh của câu đầu vào được nén thành một vectơ được chuyển vào khối giải mã. Bạn có thể thấy đây sẽ là vấn đề như thế nào đối với các câu rất dài (ví dụ: 100 mã thông báo trở lên) vì ngữ cảnh của phần đầu tiên của dữ liệu đầu vào sẽ có rất ít ảnh hưởng đến vectơ cuối cùng được truyền tới bộ giải mã.

<img src='images/plain_rnn.png'>

Việc thêm lớp chú ý vào mô hình này sẽ tránh được vấn đề này bằng cách cấp cho bộ giải mã quyền truy cập vào tất cả các phần của câu đầu vào. Để minh họa, chúng ta hãy sử dụng câu đầu vào gồm 4 từ như dưới đây. Hãy nhớ rằng trạng thái ẩn được tạo ra ở mỗi dấu thời gian của bộ mã hóa (được biểu thị bằng các hình chữ nhật màu cam). Tất cả đều được chuyển đến lớp chú ý và mỗi điểm được cho điểm dựa trên mức kích hoạt hiện tại (tức là trạng thái ẩn) của bộ giải mã. Ví dụ: hãy xem hình bên dưới nơi dự đoán đầu tiên "como" đã được đưa ra. Để đưa ra dự đoán tiếp theo, lớp chú ý trước tiên sẽ nhận tất cả các trạng thái ẩn của bộ mã hóa (tức là hình chữ nhật màu cam) cũng như trạng thái ẩn của bộ giải mã khi tạo ra từ "como" (tức là hình chữ nhật màu xanh lá cây đầu tiên). Dựa trên thông tin này, nó sẽ chấm điểm từng trạng thái ẩn của bộ mã hóa để biết bộ giải mã nên tập trung vào trạng thái nào để tạo ra từ tiếp theo. Sau quá trình đào tạo, mô hình có thể đã học được rằng nó phải căn chỉnh theo trạng thái ẩn của bộ mã hóa thứ hai và sau đó gán xác suất cao cho từ "você". Nếu chúng ta đang sử dụng giải mã tham lam, chúng ta sẽ xuất từ ​​đã nói dưới dạng ký hiệu tiếp theo, sau đó khởi động lại quá trình để tạo ra từ tiếp theo cho đến khi đạt được dự đoán ở cuối câu.

<img src='images/attention_overview.png'>


Có nhiều cách khác nhau để triển khai sự chú ý và cách chúng tôi sẽ sử dụng cho bài tập này là Chú ý sản phẩm chấm theo tỷ lệ có dạng:

$$Chú ý(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$

Bạn sẽ tìm hiểu sâu hơn về phương trình này trong tuần tới nhưng hiện tại, bạn có thể coi nó như cách tính điểm bằng cách sử dụng truy vấn (Q) và khóa (K), sau đó là phép nhân các giá trị (V) để có được vectơ ngữ cảnh tại một dấu thời gian cụ thể của bộ giải mã. Vectơ ngữ cảnh này được đưa đến bộ giải mã RNN để nhận tập hợp xác suất cho từ được dự đoán tiếp theo. Phép chia theo căn bậc hai của chiều khóa ($\sqrt{d_k}$) là để cải thiện hiệu suất mô hình và bạn cũng sẽ tìm hiểu thêm về nó vào tuần tới. Đối với ứng dụng dịch máy của chúng tôi, kích hoạt bộ mã hóa (tức là trạng thái ẩn của bộ mã hóa) sẽ là khóa và giá trị, trong khi kích hoạt bộ giải mã (tức là trạng thái ẩn của bộ giải mã) sẽ là truy vấn.

Bạn sẽ thấy trong các phần sắp tới rằng kiến ​​trúc và cơ chế phức tạp này có thể được triển khai chỉ với một vài dòng mã.

Trước tiên, bạn sẽ xác định hai biến toàn cục quan trọng:

- Kích thước của từ vựng
- Số lượng đơn vị trong các lớp LSTM (số lượng giống nhau sẽ được sử dụng cho tất cả các lớp LSTM)

Trong bài tập này, kích thước từ vựng của tiếng Anh và tiếng Bồ Đào Nha là như nhau. Do đó, chúng tôi sử dụng một hằng số VOCAB_SIZE trong toàn bộ sổ ghi chép. Mặc dù ở các bối cảnh khác, kích thước từ vựng có thể khác nhau nhưng điều đó không xảy ra trong bài tập của chúng tôi.

In [10]:
VOCAB_SIZE = 12000
UNITS = 256

<a name="ex1"></a>
## Bài tập 1 - Bộ mã hóa

Bài tập đầu tiên của bạn là mã hóa phần mã hóa của mạng lưới thần kinh. Để làm được điều này, hãy hoàn thành lớp `Bộ mã hóa` bên dưới. Lưu ý rằng trong hàm tạo (phương thức `__init__`), bạn cần xác định tất cả các lớp con của bộ mã hóa và sau đó sử dụng các lớp con này trong quá trình chuyển tiếp (phương thức `call`).

Bộ mã hóa bao gồm các lớp sau:

- [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding). Đối với lớp này, bạn cần xác định `input_dim` và `output_dim` thích hợp và cho nó biết rằng bạn đang sử dụng '0' làm phần đệm, điều này có thể được thực hiện bằng cách sử dụng giá trị thích hợp cho tham số `mask_zero`.

+ [Bidirectional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional) [Bidirectional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional). Trong TF, bạn có thể triển khai hành vi hai chiều cho các lớp giống RNN. Phần này đã được xử lý nhưng bạn sẽ cần chỉ định loại lớp thích hợp cũng như các tham số của nó. Đặc biệt, bạn cần đặt số lượng đơn vị thích hợp và đảm bảo rằng LSTM trả về chuỗi đầy đủ chứ không chỉ đầu ra cuối cùng, điều này có thể được thực hiện bằng cách sử dụng giá trị thích hợp cho tham số `return_sequences`.


Bạn cần xác định chuyển tiếp bằng cú pháp [functional API](https://www.tensorflow.org/guide/keras/functional_api) của TF. Điều này có nghĩa là bạn xâu chuỗi các lệnh gọi hàm lại với nhau để xác định mạng của mình như thế này:

```python
encoder_input = keras.Input(shape=(28, 28, 1), name="original_img")
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)
```

In [14]:
# GRADED CLASS: Encoder
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, units):
        """Initializes an instance of this class

        Args:
            vocab_size (int): Size of the vocabulary
            units (int): Number of units in the LSTM layer
        """
        super(Encoder, self).__init__()

        ### START CODE HERE ###

        self.embedding = tf.keras.layers.Embedding(  
            input_dim=vocab_size,
            output_dim=units,
            mask_zero=True
        )  

        self.rnn = tf.keras.layers.Bidirectional(  
            merge_mode="sum",  
            layer=tf.keras.layers.LSTM(
                units=units,
                return_sequences=True
            ),  
        )  

        ### END CODE HERE ###

    def call(self, context):
        """Forward pass of this layer

        Args:
            context (tf.Tensor): The sentence to translate

        Returns:
            tf.Tensor: Encoded sentence to translate
        """

        ### START CODE HERE ###

        # Pass the context through the embedding layer
        x = self.embedding(context)

        # Pass the output of the embedding through the RNN
        x = self.rnn(x)

        ### END CODE HERE ###

        return x

In [15]:
# Do a quick check of your implementation

# Create an instance of your class
encoder = Encoder(VOCAB_SIZE, UNITS)

# Pass a batch of sentences to translate from english to portuguese
encoder_output = encoder(to_translate)

print(f'Tensor of sentences in english has shape: {to_translate.shape}\n')
print(f'Encoder output has shape: {encoder_output.shape}')

Tensor of sentences in english has shape: (64, 14)

Encoder output has shape: (64, 14, 256)


##### __Kết quả mong đợi__

```
Tensor of sentences in english has shape: (64, 14)

Encoder output has shape: (64, 14, 256)
```

In [16]:
# Test your code!

w1_unittest.test_encoder(Encoder)

 All tests passed!


<a name="ex2"></a>
## Bài tập 2 - CrossAttention

Bài tập tiếp theo của bạn là mã hóa lớp sẽ thực hiện chú ý chéo giữa các câu gốc và bản dịch. Để làm được điều này, hãy hoàn thành lớp `CrossAttention` bên dưới. Lưu ý rằng trong hàm tạo (phương thức `__init__`), bạn cần xác định tất cả các lớp con và sau đó sử dụng các lớp con này trong quá trình chuyển tiếp (phương thức `call`). Đối với trường hợp cụ thể này, một số bit này đã được xử lý.

Sự chú ý chéo bao gồm các lớp sau:

- [MultiHeadAttention](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MultiHeadAttention). Đối với lớp này, bạn cần xác định `key_dim` thích hợp, đó là kích thước của các tensor khóa và truy vấn. Bạn cũng sẽ cần đặt số lượng đầu thành 1 vì bạn không triển khai sự chú ý nhiều đầu mà là sự chú ý giữa hai tensor. Lý do tại sao lớp này được ưa thích hơn [MultiHeadAttention](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MultiHeadAttention) là vì nó cho phép mã đơn giản hơn trong quá trình chuyển tiếp.

Một vài điều cần chú ý:
- Bạn cần một cách để chuyển cả đầu ra của chú ý cùng với bản dịch dịch sang phải (vì chú ý chéo này xảy ra ở phía bộ giải mã). Đối với điều này, bạn sẽ sử dụng lớp [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add) để kích thước ban đầu được giữ nguyên, điều này sẽ không xảy ra nếu bạn sử dụng thứ gì đó giống như lớp [Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add).

+ Việc chuẩn hóa lớp cũng được thực hiện để mạng ổn định hơn bằng cách sử dụng lớp [LayerNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LayerNormalization).

- Bạn không cần phải lo lắng về những bước cuối cùng này vì chúng đã được giải quyết rồi.



In [17]:
# GRADED CLASS: CrossAttention
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        """Initializes an instance of this class

        Args:
            units (int): Number of units in the LSTM layer
        """
        super().__init__()

        ### START CODE HERE ###

        self.mha = ( 
            tf.keras.layers.MultiHeadAttention(
                key_dim=units,
                num_heads=1
            ) 
        )  

        ### END CODE HERE ###

        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, context, target):
        """Forward pass of this layer

        Args:
            context (tf.Tensor): Encoded sentence to translate
            target (tf.Tensor): The embedded shifted-to-the-right translation

        Returns:
            tf.Tensor: Cross attention between context and target
        """
        ### START CODE HERE ###

        # Call the MH attention by passing in the query and value
        # For this case the query should be the translation and the value the encoded sentence to translate
        # Hint: Check the call arguments of MultiHeadAttention in the docs
        attn_output = self.mha(
            query=target,
            value=context
        )  

        ### END CODE HERE ###

        x = self.add([target, attn_output])

        x = self.layernorm(x)

        return x

In [18]:
# Do a quick check of your implementation

# Create an instance of your class
attention_layer = CrossAttention(UNITS)

# The attention layer expects the embedded sr-translation and the context
# The context (encoder_output) is already embedded so you need to do this for sr_translation:
sr_translation_embed = tf.keras.layers.Embedding(VOCAB_SIZE, output_dim=UNITS, mask_zero=True)(sr_translation)

# Compute the cross attention
attention_result = attention_layer(encoder_output, sr_translation_embed)

print(f'Tensor of contexts has shape: {encoder_output.shape}')
print(f'Tensor of translations has shape: {sr_translation_embed.shape}')
print(f'Tensor of attention scores has shape: {attention_result.shape}')

Tensor of contexts has shape: (64, 14, 256)
Tensor of translations has shape: (64, 15, 256)
Tensor of attention scores has shape: (64, 15, 256)


##### __Kết quả mong đợi__

```
Tensor of contexts has shape: (64, 14, 256)
Tensor of translations has shape: (64, 15, 256)
Tensor of attention scores has shape: (64, 15, 256)
```

In [19]:
# Test your code!

w1_unittest.test_cross_attention(CrossAttention)

 All tests passed!


<a name="ex3"></a>
## Bài tập 3 – Bộ giải mã


Bây giờ bạn sẽ triển khai phần giải mã của mạng nơ-ron bằng cách hoàn thành lớp `Bộ giải mã` bên dưới. Lưu ý rằng trong hàm tạo (phương thức `__init__`), bạn cần xác định tất cả các lớp con của bộ giải mã và sau đó sử dụng các lớp con này trong quá trình chuyển tiếp (phương thức `call`).

Bộ giải mã bao gồm các lớp sau:

- [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding). Đối với lớp này, bạn cần xác định `input_dim` và `output_dim` thích hợp và cho nó biết rằng bạn đang sử dụng '0' làm phần đệm, điều này có thể được thực hiện bằng cách sử dụng giá trị thích hợp cho tham số `mask_zero`.


+ Chú ý trước [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM). Không giống như trong bộ mã hóa mà bạn đã sử dụng LSTM hai chiều, ở đây bạn sẽ sử dụng LSTM thông thường. Đừng quên đặt số lượng đơn vị thích hợp và đảm bảo rằng LSTM trả về chuỗi đầy đủ chứ không chỉ đầu ra cuối cùng, điều này có thể được thực hiện bằng cách sử dụng giá trị thích hợp cho tham số `return_sequences`. Điều rất quan trọng là lớp này trả về trạng thái vì điều này sẽ cần thiết cho suy luận, vì vậy hãy đảm bảo đặt tham số `return_state` cho phù hợp. Lưu ý rằng các lớp LSTM trả về trạng thái dưới dạng một bộ gồm hai tensor được gọi là `memory_state` và `carry_state`, **tuy nhiên, những tên này đã được thay đổi để phản ánh tốt hơn những gì bạn đã thấy trong bài giảng thành `hidden_state` và `cell_state`** .

- Lớp chú ý thực hiện chú ý chéo giữa câu cần dịch và bản dịch dịch sang phải. Ở đây bạn cần sử dụng lớp `CrossAttention` mà bạn đã xác định trong bài tập trước.

+ Hậu chú ý [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM). Một lớp LSTM khác. Đối với cái này bạn không cần nó để trả lại trạng thái.

- Cuối cùng là lớp [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense). Cái này phải có cùng số đơn vị với kích thước của từ vựng vì bạn mong đợi nó tính toán logit cho mọi từ có thể có trong từ vựng. Đảm bảo sử dụng chức năng kích hoạt `logsoftmax` cho chức năng này, bạn có thể nhận được chức năng này dưới dạng [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).



In [25]:
# GRADED CLASS: Decoder
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, units):
        """Initializes an instance of this class

        Args:
            vocab_size (int): Size of the vocabulary
            units (int): Number of units in the LSTM layer
        """
        super(Decoder, self).__init__()

        ### START CODE HERE ###

        # The embedding layer
        self.embedding = tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=units,
            mask_zero=True
        )  

        # The RNN before attention
        self.pre_attention_rnn = tf.keras.layers.LSTM(
            units=units,
            return_sequences=True,
            return_state=True
        )  

        # The attention layer
        self.attention = CrossAttention(units)

        # The RNN after attention
        self.post_attention_rnn = tf.keras.layers.LSTM(
            units=units,
            return_sequences=True
        )  

        # The dense layer with logsoftmax activation
        self.output_layer = tf.keras.layers.Dense(
            units=vocab_size,
            activation=tf.nn.log_softmax
        )  

        ### END CODE HERE ###

    def call(self, context, target, state=None, return_state=False):
        """Forward pass of this layer

        Args:
            context (tf.Tensor): Encoded sentence to translate
            target (tf.Tensor): The shifted-to-the-right translation
            state (list[tf.Tensor, tf.Tensor], optional): Hidden state of the pre-attention LSTM. Defaults to None.
            return_state (bool, optional): If set to true return the hidden states of the LSTM. Defaults to False.

        Returns:
            tf.Tensor: The log_softmax probabilities of predicting a particular token
        """
        ### START CODE HERE ###

        # Get the embedding of the input
        x = self.embedding(target)

        # Pass the embedded input into the pre attention LSTM
        # Hints:
        # - The LSTM you defined earlier should return the output alongside the state (made up of two tensors)
        # - Pass in the state to the LSTM (needed for inference)
        x, hidden_state, cell_state = self.pre_attention_rnn(x, initial_state=state)

        # Perform cross attention between the context and the output of the LSTM (in that order)
        x = self.attention(context, x)

        # Do a pass through the post attention LSTM
        x = self.post_attention_rnn(x)

        # Compute the logits
        logits = self.output_layer(x)

        ### END CODE HERE ###

        if return_state:
            return logits, [hidden_state, cell_state]

        return logits

In [26]:
# Do a quick check of your implementation

# Create an instance of your class
decoder = Decoder(VOCAB_SIZE, UNITS)

# Notice that you don't need the embedded version of sr_translation since this is done inside the class
logits = decoder(encoder_output, sr_translation)

print(f'Tensor of contexts has shape: {encoder_output.shape}')
print(f'Tensor of right-shifted translations has shape: {sr_translation.shape}')
print(f'Tensor of logits has shape: {logits.shape}')

Tensor of contexts has shape: (64, 14, 256)
Tensor of right-shifted translations has shape: (64, 15)
Tensor of logits has shape: (64, 15, 12000)


##### __Kết quả mong đợi__

```
Tensor of contexts has shape: (64, 14, 256)
Tensor of right-shifted translations has shape: (64, 15)
Tensor of logits has shape: (64, 15, 12000)
```

In [27]:
# Test your code!

w1_unittest.test_decoder(Decoder, CrossAttention)

 All tests passed!


<a name="ex4"></a>
## Bài tập 4 – Trình dịch

Bây giờ bạn phải tập hợp tất cả các lớp mà bạn đã mã hóa trước đó thành một mô hình thực tế. Để làm điều này, hãy hoàn thành lớp `Translator` bên dưới. Lưu ý rằng không giống như các lớp Bộ mã hóa và Bộ giải mã được kế thừa từ `tf.keras.layers.Layer`, lớp Translator kế thừa từ `tf.keras.Model`.

Hãy nhớ rằng `train_data` sẽ tạo ra một bộ dữ liệu có câu cần dịch và bản dịch được dịch sang phải, đây là những "đặc điểm" của mô hình. Điều này có nghĩa là đầu vào của mạng của bạn sẽ là các bộ chứa ngữ cảnh và mục tiêu.

In [28]:
# GRADED CLASS: Translator
class Translator(tf.keras.Model):
    def __init__(self, vocab_size, units):
        """Initializes an instance of this class

        Args:
            vocab_size (int): Size of the vocabulary
            units (int): Number of units in the LSTM layer
        """
        super().__init__()

        ### START CODE HERE ###

        # Define the encoder with the appropriate vocab_size and number of units
        self.encoder = Encoder(vocab_size, units)

        # Define the decoder with the appropriate vocab_size and number of units
        self.decoder = Decoder(vocab_size, units)

        ### END CODE HERE ###

    def call(self, inputs):
        """Forward pass of this layer

        Args:
            inputs (tuple(tf.Tensor, tf.Tensor)): Tuple containing the context (sentence to translate) and the target (shifted-to-the-right translation)

        Returns:
            tf.Tensor: The log_softmax probabilities of predicting a particular token
        """

        ### START CODE HERE ###

        # In this case inputs is a tuple consisting of the context and the target, unpack it into single variables
        context, target = inputs

        # Pass the context through the encoder
        encoded_context = self.encoder(context)

        # Compute the logits by passing the encoded context and the target to the decoder
        logits = self.decoder(encoded_context, target)

        ### END CODE HERE ###

        return logits

In [29]:
# Do a quick check of your implementation

# Create an instance of your class
translator = Translator(VOCAB_SIZE, UNITS)

# Compute the logits for every word in the vocabulary
logits = translator((to_translate, sr_translation))

print(f'Tensor of sentences to translate has shape: {to_translate.shape}')
print(f'Tensor of right-shifted translations has shape: {sr_translation.shape}')
print(f'Tensor of logits has shape: {logits.shape}')

Tensor of sentences to translate has shape: (64, 14)
Tensor of right-shifted translations has shape: (64, 15)
Tensor of logits has shape: (64, 15, 12000)


##### __Kết quả mong đợi__

```
Tensor of sentences to translate has shape: (64, 14)
Tensor of right-shifted translations has shape: (64, 15)
Tensor of logits has shape: (64, 15, 12000)
```

In [30]:
w1_unittest.test_translator(Translator, Encoder, Decoder)

 All tests passed!


<a tên="3"></a>
## 3. Đào tạo

Bây giờ bạn đã có một phiên bản chưa được huấn luyện của mô hình NMT, đã đến lúc huấn luyện nó. Bạn có thể sử dụng hàm `compile_and_train` bên dưới để đạt được điều này:

In [31]:
def compile_and_train(model, epochs=20, steps_per_epoch=500):
    model.compile(optimizer="adam", loss=masked_loss, metrics=[masked_acc, masked_loss])

    history = model.fit(
        train_data.repeat(),
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_data,
        validation_steps=50,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)],
    )

    return model, history

In [32]:
# Train the translator (this takes some minutes so feel free to take a break)

trained_translator, history = compile_and_train(translator)

Epoch 1/20
500/500 [==============================] - 48s 68ms/step - loss: 5.1730 - masked_acc: 0.2171 - masked_loss: 5.1756 - val_loss: 4.4001 - val_masked_acc: 0.3205 - val_masked_loss: 4.4010
Epoch 2/20
500/500 [==============================] - 17s 33ms/step - loss: 3.8393 - masked_acc: 0.4030 - masked_loss: 3.8401 - val_loss: 3.1249 - val_masked_acc: 0.4911 - val_masked_loss: 3.1270
Epoch 3/20
500/500 [==============================] - 16s 32ms/step - loss: 2.8200 - masked_acc: 0.5347 - masked_loss: 2.8215 - val_loss: 2.4602 - val_masked_acc: 0.5754 - val_masked_loss: 2.4607
Epoch 4/20
500/500 [==============================] - 16s 32ms/step - loss: 2.2937 - masked_acc: 0.6065 - masked_loss: 2.2946 - val_loss: 2.0259 - val_masked_acc: 0.6419 - val_masked_loss: 2.0265
Epoch 5/20
500/500 [==============================] - 16s 31ms/step - loss: 1.9214 - masked_acc: 0.6588 - masked_loss: 1.9226 - val_loss: 1.7629 - val_masked_acc: 0.6764 - val_masked_loss: 1.7635
Epoch 6/20
500/500 [

<a tên="4"></a>
## 4. Sử dụng mô hình để suy luận


Bây giờ mô hình của bạn đã được đào tạo, bạn có thể sử dụng nó để suy luận. Để giúp bạn thực hiện việc này, chức năng `generate_next_token` được cung cấp. Lưu ý rằng hàm này được sử dụng bên trong vòng lặp for, do đó bạn cung cấp cho nó thông tin của bước trước để tạo thông tin cho bước tiếp theo. Đặc biệt, bạn cần theo dõi trạng thái của LSTM chú ý trước trong bộ giải mã và liệu bạn đã hoàn thành việc dịch hay chưa. Cũng lưu ý rằng biến `nhiệt độ` được đưa ra để xác định cách chọn mã thông báo tiếp theo dựa trên nhật ký dự đoán:

In [33]:
def generate_next_token(decoder, context, next_token, done, state, temperature=0.0):
    """Generates the next token in the sequence

    Args:
        decoder (Decoder): The decoder
        context (tf.Tensor): Encoded sentence to translate
        next_token (tf.Tensor): The predicted next token
        done (bool): True if the translation is complete
        state (list[tf.Tensor, tf.Tensor]): Hidden states of the pre-attention LSTM layer
        temperature (float, optional): The temperature that controls the randomness of the predicted tokens. Defaults to 0.0.

    Returns:
        tuple(tf.Tensor, np.float, list[tf.Tensor, tf.Tensor], bool): The next token, log prob of said token, hidden state of LSTM and if translation is done
    """
    # Get the logits and state from the decoder
    logits, state = decoder(context, next_token, state=state, return_state=True)
    
    # Trim the intermediate dimension 
    logits = logits[:, -1, :]
        
    # If temp is 0 then next_token is the argmax of logits
    if temperature == 0.0:
        next_token = tf.argmax(logits, axis=-1)
        
    # If temp is not 0 then next_token is sampled out of logits
    else:
        logits = logits / temperature
        next_token = tf.random.categorical(logits, num_samples=1)
    
    # Trim dimensions of size 1
    logits = tf.squeeze(logits)
    next_token = tf.squeeze(next_token)
    
    # Get the logit of the selected next_token
    logit = logits[next_token].numpy()
    
    # Reshape to (1,1) since this is the expected shape for text encoded as TF tensors
    next_token = tf.reshape(next_token, shape=(1,1))
    
    # If next_token is End-of-Sentence token you are done
    if next_token == eos_id:
        done = True
    
    return next_token, logit, state, done

Xem cách nó hoạt động bằng cách chạy ô sau:

In [35]:
# PROCESS SENTENCE TO TRANSLATE AND ENCODE

# A sentence you wish to translate
eng_sentence = "I love languages"

# Convert it to a tensor
texts = tf.convert_to_tensor(eng_sentence)[tf.newaxis]

# Vectorize it and pass it through the encoder
context = english_vectorizer(texts).to_tensor()
context = encoder(context)

# SET STATE OF THE DECODER

# Next token is Start-of-Sentence since you are starting fresh
next_token = tf.fill((1,1), sos_id)

# Hidden and Cell states of the LSTM can be mocked using uniform samples
state = [tf.random.uniform((1, UNITS)), tf.random.uniform((1, UNITS))]

# You are not done until next token is EOS token
done = False

# Generate next token
next_token, logit, state, done = generate_next_token(decoder, context, next_token, done, state, temperature=0.5)
print(f"Next token: {next_token}\nLogit: {logit:.4f}\nDone? {done}")

Next token: [[10026]]
Logit: -18.8343
Done? False


<a name="ex5"></a>
## Bài tập 5 - dịch

Bây giờ bạn có thể kết hợp mọi thứ lại với nhau để dịch một câu nhất định. Để làm điều này, hãy hoàn thành chức năng `translate` bên dưới. Chức năng này sẽ đảm nhiệm các bước sau:
- Xử lý câu để dịch và mã hóa nó

+ Đặt trạng thái ban đầu của bộ giải mã

- Nhận dự đoán về mã thông báo tiếp theo (bắt đầu bằng mã thông báo \<SOS>) cho số lần lặp tối đa (trong trường hợp mã thông báo \<EOS> không bao giờ được trả lại)

+ Trả về văn bản đã dịch (dưới dạng chuỗi), logit của lần lặp cuối cùng (điều này giúp đo lường mức độ chắc chắn rằng toàn bộ chuỗi đã được dịch) và bản dịch ở định dạng mã thông báo.


Gợi ý:

- Ô trước cung cấp nhiều thông tin chi tiết về cách hoạt động của chức năng này, vì vậy nếu bạn gặp khó khăn, hãy tham khảo ô đó.

+ Một số tài liệu hữu ích:
+ [tf.newaxis](https://www.tensorflow.org/api_docs/python/tf#newaxis)

- [tf.fill](https://www.tensorflow.org/api_docs/python/tf/fill)

+ [tf.zeros](https://www.tensorflow.org/api_docs/python/tf/zeros)


In [49]:
# GRADED FUNCTION: translate
def translate(model, text, max_length=50, temperature=0.0):
    """Translate a given sentence from English to Portuguese

    Args:
        model (tf.keras.Model): The trained translator
        text (string): The sentence to translate
        max_length (int, optional): The maximum length of the translation. Defaults to 50.
        temperature (float, optional): The temperature that controls the randomness of the predicted tokens. Defaults to 0.0.

    Returns:
        tuple(str, np.float, tf.Tensor): The translation, logit that predicted <EOS> token and the tokenized translation
    """
    # Lists to save tokens and logits
    tokens, logits = [], []

    ### START CODE HERE ###
    
    # PROCESS THE SENTENCE TO TRANSLATE
    
    # Convert the original string into a tensor
    text = tf.convert_to_tensor(text)[tf.newaxis]
    
    # Vectorize the text using the correct vectorizer
    context = english_vectorizer(text).to_tensor()
    
    # Get the encoded context (pass the context through the encoder)
    # Hint: Remember you can get the encoder by using model.encoder
    context = model.encoder(context)
    
    # INITIAL STATE OF THE DECODER
    
    # First token should be SOS token with shape (1,1)
    next_token = tf.fill((1, 1), sos_id)
    
    # Initial hidden and cell states should be tensors of zeros with shape (1, UNITS)
    state = [tf.zeros((1, UNITS)), tf.zeros((1, UNITS))]
    
    # You are done when you draw a EOS token as next token (initial state is False)
    done = False

    # Iterate for max_length iterations
    for _ in range(max_length):
        # Generate the next token
        try:
            next_token, logit, state, done = generate_next_token(
                decoder=model.decoder,
                context=context,
                next_token=next_token,
                done=done,
                state=state,
                temperature=temperature
            )
        except:
             raise Exception("Problem generating the next token")
        
        # If done then break out of the loop
        if done:
            break
        
        # Add next_token to the list of tokens
        tokens.append(next_token)
        
        # Add logit to the list of logits
        logits.append(logit)
    
    ### END CODE HERE ###
    
    # Concatenate all tokens into a tensor
    tokens = tf.concat(tokens, axis=-1)
    
    # Convert the translated tokens into text
    translation = tf.squeeze(tokens_to_text(tokens, id_to_word))
    translation = translation.numpy().decode()
    
    return translation, logits[-1], tokens

Hãy thử hàm của bạn với nhiệt độ bằng 0, hàm này sẽ mang lại đầu ra xác định và tương đương với giải mã tham lam:

In [50]:
# Running this cell multiple times should return the same output since temp is 0

temp = 0.0 
original_sentence = "I love languages"

translation, logit, tokens = translate(trained_translator, original_sentence, temperature=temp)

print(f"Temperature: {temp}\n\nOriginal sentence: {original_sentence}\nTranslation: {translation}\nTranslation tokens:{tokens}\nLogit: {logit:.3f}")

Temperature: 0.0

Original sentence: I love languages
Translation: eu amo idiomas voce tem chovido .
Translation tokens:[[   9  522  850   14   33 4211    4]]
Logit: -0.403


Hãy thử chức năng của bạn với nhiệt độ 0,7 (đầu ra ngẫu nhiên):

In [51]:
# Running this cell multiple times should return different outputs since temp is not 0
# You can try different temperatures

temp = 0.7
original_sentence = "I love languages"

translation, logit, tokens = translate(trained_translator, original_sentence, temperature=temp)

print(f"Temperature: {temp}\n\nOriginal sentence: {original_sentence}\nTranslation: {translation}\nTranslation tokens:{tokens}\nLogit: {logit:.3f}")

Temperature: 0.7

Original sentence: I love languages
Translation: eu eu amo idiomas ainda estao famosos .
Translation tokens:[[   9    9  522  850   61   71 4140    4]]
Logit: -0.353


In [52]:
w1_unittest.test_translate(translate, trained_translator)

 All tests passed!


<a tên="5"></a>
## 5. Giải mã rủi ro Bayes tối thiểu

Như đã đề cập trong các bài giảng, việc nhận được mã thông báo có khả năng xảy ra cao nhất ở mỗi bước có thể không nhất thiết mang lại kết quả tốt nhất. Một cách tiếp cận khác là thực hiện Giải mã rủi ro Bayes tối thiểu hoặc MBR. Các bước chung để thực hiện điều này là:

- Lấy một số mẫu ngẫu nhiên
+ Cho điểm từng mẫu so với tất cả các mẫu khác
- Chọn người có số điểm cao nhất

Bạn sẽ xây dựng các hàm trợ giúp cho các bước này trong các phần sau.

Với khả năng tạo ra các bản dịch khác nhau bằng cách đặt các giá trị nhiệt độ khác nhau, bạn có thể thực hiện những gì bạn đã thấy trong bài giảng và tạo ra một loạt bản dịch, sau đó xác định bản dịch nào là phù hợp nhất. Bây giờ bạn sẽ thực hiện việc này bằng cách sử dụng hàm `generate_samples` được cung cấp. Hàm này sẽ trả về bất kỳ số lượng bản dịch ứng cử viên mong muốn nào cùng với xác suất ghi nhật ký cho mỗi bản dịch:

In [53]:
def generate_samples(model, text, n_samples=4, temperature=0.6):
    
    samples, log_probs = [], []

    # Iterate for n_samples iterations
    for _ in range(n_samples):
        
        # Save the logit and the translated tensor
        _, logp, sample = translate(model, text, temperature=temperature)
        
        # Save the translated tensors
        samples.append(np.squeeze(sample.numpy()).tolist())
        
        # Save the logits
        log_probs.append(logp)
                
    return samples, log_probs

In [54]:
samples, log_probs = generate_samples(trained_translator, 'I love languages')

for s, l in zip(samples, log_probs):
    print(f"Translated tensor: {s} has logit: {l:.3f}")

Translated tensor: [9, 522, 850, 14, 33, 1232, 8115, 4] has logit: -1.212
Translated tensor: [9, 522, 850, 14, 33, 156, 304, 4] has logit: -0.798
Translated tensor: [9, 255, 850, 12, 256, 10] has logit: -0.375
Translated tensor: [9, 811, 850, 14, 33, 149, 4] has logit: -0.441


## So sánh sự trùng lặp

Bây giờ bạn có thể tạo nhiều bản dịch, đã đến lúc nghĩ ra một phương pháp để đo lường mức độ tốt của từng bản dịch. Như bạn đã thấy trong các bài giảng, một cách để đạt được điều này là so sánh từng mẫu với các mẫu khác.

Có một số số liệu bạn có thể sử dụng cho mục đích này, như được trình bày trong các bài giảng và bạn có thể thử thử nghiệm bất kỳ số liệu nào trong số này. Đối với bài tập này, bạn sẽ tính điểm cho **sự trùng lặp unigram**.

Một trong những số liệu này là [Jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index) được sử dụng rộng rãi nhưng đơn giản, có giao điểm trên sự kết hợp của hai bộ. Hàm `jaccard_similarity` trả về số liệu này cho bất kỳ cặp bản dịch ứng viên và bản dịch tham chiếu nào:


In [55]:
def jaccard_similarity(candidate, reference):
        
    # Convert the lists to sets to get the unique tokens
    candidate_set = set(candidate)
    reference_set = set(reference)
    
    # Get the set of tokens common to both candidate and reference
    common_tokens = candidate_set.intersection(reference_set)
    
    # Get the set of all tokens found in either candidate or reference
    all_tokens = candidate_set.union(reference_set)
    
    # Compute the percentage of overlap (divide the number of common tokens by the number of all tokens)
    overlap = len(common_tokens) / len(all_tokens)
        
    return overlap

In [56]:
l1 = [1, 2, 3]
l2 = [1, 2, 3, 4]

js = jaccard_similarity(l1, l2)

print(f"jaccard similarity between lists: {l1} and {l2} is {js:.3f}")

jaccard similarity between lists: [1, 2, 3] and [1, 2, 3, 4] is 0.750


##### __Kết quả mong đợi__

```
jaccard similarity between tensors: [1, 2, 3] and [1, 2, 3, 4] is 0.750

```

<a name="ex6"></a>
## Bài tập 6 - rouge1_similarity

Độ tương tự của Jaccard là tốt nhưng thước đo được sử dụng phổ biến hơn trong dịch máy là điểm ROUGE. Đối với unigram, cái này được gọi là ROUGE-1 và như được trình bày trong bài giảng, bạn có thể xuất điểm cho cả độ chính xác và khả năng thu hồi khi so sánh hai mẫu. Để có được điểm số cuối cùng, bạn sẽ muốn tính điểm F1 như sau:

$$score = 2* \frac{(độ chính xác * thu hồi)}{(độ chính xác + thu hồi)}$$

Để triển khai hàm `rouge1_similarity`, bạn muốn sử dụng lớp [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) từ thư viện chuẩn Python:

In [57]:
# GRADED FUNCTION: rouge1_similarity
def rouge1_similarity(candidate, reference):
    """Computes the ROUGE 1 score between two token lists

    Args:
        candidate (list[int]): Tokenized candidate translation
        reference (list[int]): Tokenized reference translation

    Returns:
        float: Overlap between the two token lists
    """
    ### START CODE HERE ###
    
    # Make a frequency table of the candidate and reference tokens
    # Hint: use the Counter class (already imported)
    candidate_word_counts = Counter(candidate)
    reference_word_counts = Counter(reference)
    
    # Initialize overlap at 0
    overlap = 0
    
    # Iterate over the tokens in the candidate frequency table
    # Hint: Counter is a subclass of dict and you can get the keys 
    #       out of a dict using the keys method like this: dict.keys()
    for token in candidate_word_counts.keys():
        
        # Get the count of the current token in the candidate frequency table
        # Hint: You can access the counts of a token as you would access values of a dictionary
        token_count_candidate = candidate_word_counts[token]
        
        # Get the count of the current token in the reference frequency table
        # Hint: You can access the counts of a token as you would access values of a dictionary
        token_count_reference = reference_word_counts[token]
        
        # Update the overlap by getting the minimum between the two token counts above
        overlap += min(token_count_candidate, token_count_reference)
    
    # Compute the precision
    # Hint: precision = overlap / (number of tokens in candidate list) 
    precision = overlap / len(candidate)
    
    # Compute the recall
    # Hint: recall = overlap / (number of tokens in reference list) 
    recall = overlap / len(reference)
    
    if precision + recall != 0:
        # Compute the Rouge1 Score
        # Hint: This is equivalent to the F1 score
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        return f1_score
    
    ### END CODE HERE ###
        
    return 0 # If precision + recall = 0 then return 0

In [58]:
l1 = [1, 2, 3]
l2 = [1, 2, 3, 4]

r1s = rouge1_similarity(l1, l2)

print(f"rouge 1 similarity between lists: {l1} and {l2} is {r1s:.3f}")

rouge 1 similarity between lists: [1, 2, 3] and [1, 2, 3, 4] is 0.857


##### __Kết quả mong đợi__

```
rouge 1 similarity between lists: [1, 2, 3] and [1, 2, 3, 4] is 0.857

```

In [59]:
w1_unittest.test_rouge1_similarity(rouge1_similarity)

 All tests passed!


## Tính tổng điểm


Bây giờ bạn sẽ xây dựng một hàm để tạo ra điểm tổng thể cho một mẫu cụ thể. Như đã đề cập trong bài giảng, bạn cần so sánh từng mẫu với tất cả các mẫu khác. Ví dụ: nếu tạo ra 30 câu, chúng ta sẽ cần so sánh câu 1 với câu 2 đến câu 30. Sau đó, chúng ta so sánh câu 2 với câu 1 và 3 đến câu 30, v.v. Ở mỗi bước, chúng tôi lấy điểm trung bình của tất cả các so sánh để có được điểm tổng thể cho một mẫu cụ thể. Để minh họa, đây sẽ là các bước để tạo điểm của danh sách 4 mẫu.

- Lấy điểm tương đồng giữa mẫu 1 và mẫu 2
+ Lấy điểm tương đồng giữa mẫu 1 và mẫu 3
- Lấy điểm tương đồng giữa mẫu 1 và mẫu 4
+ Lấy điểm trung bình của 3 bước đầu tiên. Đây sẽ là tổng điểm của mẫu 1
- Lặp lại cho đến khi mẫu từ 1 đến 4 có tổng điểm.


Kết quả sẽ được lưu vào từ điển để dễ dàng tra cứu.

<a name="ex7"></a>
## Bài tập 7 - Average_overlap

Hoàn thành hàm `average_overlap` bên dưới để triển khai quy trình được mô tả ở trên:

In [62]:
# GRADED FUNCTION: average_overlap
def average_overlap(samples, similarity_fn):
    """Computes the arithmetic mean of each candidate sentence in the samples

    Args:
        samples (list[list[int]]): Tokenized version of translated sentences
        similarity_fn (Function): Similarity function used to compute the overlap

    Returns:
        dict[int, float]: A dictionary mapping the index of each translation to its score
    """
    # Initialize dictionary
    scores = {}
    
    # Iterate through all samples (enumerate helps keep track of indexes)
    for index_candidate, candidate in enumerate(samples):    
        
        ### START CODE HERE ###
                
        # Initially overlap is zero
        overlap = 0
        
        # Iterate through all samples (enumerate helps keep track of indexes)
        for index_sample, sample in enumerate(samples):

            # Skip if the candidate index is the same as the sample index
            if index_candidate == index_sample:
                continue
                
            # Get the overlap between candidate and sample using the similarity function
            sample_overlap = similarity_fn(candidate, sample)
            
            # Add the sample overlap to the total overlap
            overlap += sample_overlap

        ### END CODE HERE ###
        
        # Get the score for the candidate by computing the average
        score = overlap / (len(samples) - 1)

        # Only use 3 decimal points
        score = round(score, 3)
        
        # Save the score in the dictionary. use index as the key.
        scores[index_candidate] = score
        
    return scores

In [63]:
# Test with Jaccard similarity

l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]

avg_ovlp = average_overlap([l1, l2, l3], jaccard_similarity)

print(f"average overlap between lists: {l1}, {l2} and {l3} using Jaccard similarity is:\n\n{avg_ovlp}")

average overlap between lists: [1, 2, 3], [1, 2, 4] and [1, 2, 4, 5] using Jaccard similarity is:

{0: 0.45, 1: 0.625, 2: 0.575}


##### __Kết quả mong đợi__

```
average overlap between lists: [1, 2, 3], [1, 2, 4] and [1, 2, 4, 5] using Jaccard similarity is:

{0: 0.45, 1: 0.625, 2: 0.575}
```

In [64]:
# Test with Rouge1 similarity

l1 = [1, 2, 3]
l2 = [1, 4]
l3 = [1, 2, 4, 5]
l4 = [5,6]

avg_ovlp = average_overlap([l1, l2, l3, l4], rouge1_similarity)

print(f"average overlap between lists: {l1}, {l2}, {l3} and {l4} using Rouge1 similarity is:\n\n{avg_ovlp}")

average overlap between lists: [1, 2, 3], [1, 4], [1, 2, 4, 5] and [5, 6] using Rouge1 similarity is:

{0: 0.324, 1: 0.356, 2: 0.524, 3: 0.111}


##### __Kết quả mong đợi__

```
average overlap between lists: [1, 2, 3], [1, 4], [1, 2, 4, 5] and [5, 6] using Rouge1 similarity is:

{0: 0.324, 1: 0.356, 2: 0.524, 3: 0.111}
```

In [65]:
w1_unittest.test_average_overlap(average_overlap)

 All tests passed!


Trong thực tế, người ta cũng thường thấy giá trị trung bình có trọng số được sử dụng để tính điểm tổng thể thay vì chỉ giá trị trung bình số học. Điều này được triển khai trong hàm `weighted_avg_overlap` bên dưới và bạn có thể sử dụng nó trong các thử nghiệm của mình để xem cái nào sẽ cho kết quả tốt hơn:

In [66]:
def weighted_avg_overlap(samples, log_probs, similarity_fn):
    
    # Scores dictionary
    scores = {}
    
    # Iterate over the samples
    for index_candidate, candidate in enumerate(samples):    
        
        # Initialize overlap and weighted sum
        overlap, weight_sum = 0.0, 0.0
        
        # Iterate over all samples and log probabilities
        for index_sample, (sample, logp) in enumerate(zip(samples, log_probs)):

            # Skip if the candidate index is the same as the sample index            
            if index_candidate == index_sample:
                continue
                
            # Convert log probability to linear scale
            sample_p = float(np.exp(logp))

            # Update the weighted sum
            weight_sum += sample_p

            # Get the unigram overlap between candidate and sample
            sample_overlap = similarity_fn(candidate, sample)
            
            # Update the overlap
            overlap += sample_p * sample_overlap
            
        # Compute the score for the candidate
        score = overlap / weight_sum

        # Only use 3 decimal points
        score = round(score, 3)
        
        # Save the score in the dictionary. use index as the key.
        scores[index_candidate] = score
    
    return scores

In [67]:
l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]
log_probs = [0.4, 0.2, 0.5]

w_avg_ovlp = weighted_avg_overlap([l1, l2, l3], log_probs, jaccard_similarity)

print(f"weighted average overlap using Jaccard similarity is:\n\n{w_avg_ovlp}")

weighted average overlap using Jaccard similarity is:

{0: 0.443, 1: 0.631, 2: 0.558}


## mbr_decode

Bây giờ bạn sẽ tập hợp mọi thứ lại với nhau trong hàm `mbr_decode` bên dưới. Bước cuối cùng này không được chấm điểm vì hàm này chỉ là phần bao bọc xung quanh tất cả nội dung thú vị mà bạn đã mã hóa cho đến nay!

Bạn có thể sử dụng nó để thử nghiệm, thử số lượng mẫu, nhiệt độ và hàm tương tự khác nhau!

In [69]:
def mbr_decode(model, text, n_samples=5, temperature=0.6, similarity_fn=jaccard_similarity):
    
    # Generate samples
    samples, log_probs = generate_samples(model, text, n_samples=n_samples, temperature=temperature)
    
    # Compute the overlap scores
    scores = weighted_avg_overlap(samples, log_probs, similarity_fn)

    # Decode samples
    decoded_translations = [tokens_to_text(s, id_to_word).numpy().decode('utf-8') for s in samples]
    
    # Find the key with the highest score
    max_score_key = max(scores, key=lambda k: scores[k])
    
    # Get the translation 
    translation = decoded_translations[max_score_key]
    
    return translation, decoded_translations

In [70]:
english_sentence = "I love languages"

translation, candidates = mbr_decode(trained_translator, english_sentence, n_samples=10, temperature=0.6)

print("Translation candidates:")
for c in candidates:
    print(c)

print(f"\nSelected translation: {translation}")

Translation candidates:
eu amo idiomas voce tem alguma estante de tempo . ! eu eu planejo .
eu amo idiomas voce tem que ?
eu eu amo idiomas voce e de pao .
eu amor idiomas a gente ?
eu adoro idiomas ?
eu amo idiomas voces de vitela .
eu amo idiomas voce tem comida .
eu adorei idiomas ?
eu adoro idiomas voce .
eu eu ama idiomas voce tem certeza .

Selected translation: eu amo idiomas voce tem comida .


**Xin chúc mừng!** Tuần tới, bạn sẽ tìm hiểu sâu hơn về các mô hình chú ý và nghiên cứu kiến ​​trúc Máy biến áp. Bạn sẽ xây dựng một mạng khác nhưng không có phần lặp lại. Nó sẽ cho thấy rằng sự chú ý là tất cả những gì bạn cần! Thật thú vị!

**Hãy tiếp tục phát huy!**